In [1]:
%cd ../..

E:\システムトレード入門\trade_system_git_workspace


In [2]:
import pandas as pd
import numpy as np
from pytz import timezone
import datetime
from pathlib import Path
from tqdm.notebook import tqdm

In [3]:
from get_stock_price import StockDatabase

In [4]:
import py_workdays

In [5]:
from portfolio.trade_transformer import PortfolioTransformer, PortfolioRestrictorIdentity, FeeCalculatorFree
from portfolio.price_supply import StockDBPriceSupplier, CannotGetAllDataError

In [6]:
from utils import get_next_workday_intraday_datetime

In [7]:
jst = timezone("Asia/Tokyo")
start_datetime = jst.localize(datetime.datetime(2020,11,10,0,0,0))
end_datetime = jst.localize(datetime.datetime(2020,11,20,0,0,0))
freq_str = "5T"
episode_length = 300

### 開始時刻とエピソードのリストを求める

In [8]:
start_datetime_list = []
episode_length_list = []

iter_datetime = start_datetime
start_datetime_list.append(iter_datetime)
episode_length_list.append(episode_length)

while True:
    iter_datetime = get_next_workday_intraday_datetime(iter_datetime, freq_str, number=episode_length)
    start_datetime_list.append(iter_datetime)
    if iter_datetime > end_datetime:
        date_array = py_workdays.extract_workdays_intraday_jp_index(pd.date_range(pre_start_datetime, end_datetime, freq=freq_str, closed="left"))
        episode_length_list.append(len(date_array))
        
        # 範囲の最後
        start_datetime_list.append(end_datetime)
        episode_length_list.append(episode_length)
        
        break
        
    episode_length_list.append(episode_length)
    pre_start_datetime = iter_datetime
        

In [9]:
start_datetime_list

[datetime.datetime(2020, 11, 10, 0, 0, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>),
 datetime.datetime(2020, 11, 17, 9, 0, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>),
 datetime.datetime(2020, 11, 25, 9, 0, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>),
 datetime.datetime(2020, 11, 20, 0, 0, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>)]

In [10]:
episode_length_list

[300, 300, 180, 300]

In [11]:
db_path = Path("db/sub_stock_db/nikkei_255_stock.db")

ticker_codes_df = pd.read_csv(Path("portfolio/rl_base/nikkei225.csv"), header=0)  # 自分で作成
ticker_codes = ticker_codes_df["code"].values.astype(str).tolist()

In [12]:
len(ticker_codes)

219

In [13]:
# stock_db
stock_db = StockDatabase(db_path)

# PriceSupplierの設定
price_supplier = StockDBPriceSupplier(stock_db,
                                      [],  # 最初は何の銘柄コードも指定しない
                                      None,
                                      freq_str,
                                      interpolate=True
                                     )


bool_list = []

for ticker_code in tqdm(ticker_codes):
    price_supplier.ticker_names = list([ticker_code])
    try:
        for one_start_datetime, one_episode_length in zip(start_datetime_list, episode_length_list):
            price_supplier.episode_length = one_episode_length
            price_supplier.reset(one_start_datetime)
                
        bool_list.append(True)
    except:
        bool_list.append(False)

In [14]:
bool_array = np.array(bool_list)
bool_array

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [15]:
save_df = ticker_codes_df.loc[bool_array,:]
save_df.to_csv(Path("portfolio/rl_base/nikkei225_modified.csv"), index=False, header=True)